<a href="https://colab.research.google.com/github/gregory6-626/Telescopes-and-Photometry/blob/main/Photometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  1.Import necessary modules

In [ ]:
pip install gdown

In [ ]:
pip install photutils

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.wcs import WCS
import pandas as pd
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from astropy.stats import mad_std
from astropy.visualization import simple_norm
import matplotlib.patches as patches
from astropy.stats import sigma_clipped_stats
plt.style.use(astropy_mpl_style)
from photutils.aperture import ApertureStats
import numpy as np
import gdown

#  2.Load & Display the image file

In [ ]:


file_id = '1_Cm1B82WMI48rRnZtDzAllkH0fU5n7og'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
output = 'image_file.fits'  # Provide a local file name for the downloaded FITS file
gdown.download(url, output, quiet=False)

In [ ]:
image_file =fits.open('image_file.fits') # here I have given the JWST NIRCAM F335 image of Carina Nebula
image_file.info()

In [ ]:
header=image_file[0].header
image=image_file[0].data

In [ ]:
# show the image
fig, ax = plt.subplots(figsize=(40,25))
norm = simple_norm(image, 'sqrt', percent=99.5)
ax.tick_params(axis='x', labelsize=40)
ax.tick_params(axis='y', labelsize=40)
ax.set_title("NGC 3324(Carina Nebula)- JWST NIRCam F335",size=40)
ax.imshow(image,norm=norm,interpolation='nearest')
ax.add_patch(rect)
ax.invert_yaxis()
plt.grid(False)

## 2.1 Select the Region of interest (ROI)

In [ ]:
x1=                        # give the desired x and y coordinates
x2=
y1=
y2=

In [ ]:
roi=image[y1:y2,x1:x2] # select the region

In [ ]:
fig, ax = plt.subplots(figsize=(40,25))
norm = simple_norm(image, 'sqrt', percent=99.5)
rect=patches.Rectangle((x1,y1),x2-x1,y2-y1,linewidth=5,edgecolor='b',facecolor='none')
ax.tick_params(axis='x', labelsize=40)
ax.tick_params(axis='y', labelsize=40)
ax.set_title("NGC 3324(Carina Nebula)- JWST NIRCam F335",size=40)
ax.imshow(image,norm=norm,interpolation='nearest')
ax.add_patch(rect)
ax.invert_yaxis()
plt.grid(False)

In [ ]:
plt.figure(figsize=(10,10)) #show the region
plt.imshow(roi,norm=norm)
plt.gca().invert_yaxis()
plt.grid(False)

# 3.Background subtraction

In [ ]:
mean, median, std = sigma_clipped_stats(roi, sigma=3.0, cenfunc='median', stdfunc='std')
print(median,std)
corrected_roi=roi-median # subtract the median
plt.figure(figsize=(10,10)) #show the background subtracted image
plt.imshow(corrected_roi,norm=norm)
plt.gca().invert_yaxis()
plt.grid(False)


# 4.Identify the Sources

In [ ]:
daofind = DAOStarFinder(fwhm=3, threshold=5*std) # you can change the fwhm and threshold limits
sources = daofind(corrected_roi)
for col in sources.colnames:
	sources[col].info.format = '%.9g'
print(sources)


In [ ]:
position = [(x, y) for x, y in zip(sources['xcentroid'], sources['ycentroid'])]
apertures = CircularAperture(position, r=9) # put the aperture around the sources, you can change the radius(preferably radius > 3*fwhm)

In [ ]:
# Display the identified sources
plt.figure(figsize=(10,10))
plt.imshow(corrected_roi, norm=norm, interpolation='nearest')
ap_patch = apertures.plot(color='white', lw=1.5,
                           label='Photometry aperture')
plt.gca().invert_yaxis()
plt.grid(False)


#  5.Performing Aperture Photometry

In [ ]:
phot_table = aperture_photometry(corrected_roi, apertures) # performing photometry
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

## 5.1 Fitting annulus(to remove background fluxes)

In [ ]:
annulus_aperture = CircularAnnulus(position, r_in=12, r_out=15) # putting the annulus , you can change r_in & r_out

In [ ]:
#to show the annular ring and aperture along with the source
plt.figure(figsize=(5,5))
plt.imshow(corrected_roi,norm=norm, interpolation='nearest')
ap_patch = apertures.plot(color='white', lw=2)
ann_patches = annulus_aperture.plot(color='blue', lw=2)
#plt.ylim(43,100) #give as you need and remove the hash(#)
#plt.xlim(0, 100)
plt.grid(False)

In [ ]:
aperstats = ApertureStats(corrected_roi, annulus_aperture) #calculating the mean flux of the each aperture
bkg_mean = aperstats.mean
print(bkg_mean)

## 5.2 Correcting Aperture Sum

In [ ]:
area=apertures.area # calculate the area of each aperture

In [ ]:
total_bkg = bkg_mean * area # calculate the bkg flux inside that area

In [ ]:
corrected_sum=phot_table['aperture_sum'] - total_bkg

In [ ]:
phot_table['Corrected_sum']=corrected_sum

In [ ]:
phot_table

# 6 Converting x,y coordinates into RA & DEC

In [ ]:
df=phot_table.to_pandas() # for easier calculation

In [ ]:
df=df[df['Corrected_sum']>0] # to remove negative sum

In [ ]:
df['X']=df['xcenter']+x1  #to get the original coordinates
df['Y']=df['ycenter']+y1

In [ ]:
wcs=WCS(image_file[0].header)

In [ ]:
x,y=wcs.pixel_to_world_values(df['X'],df['Y'])

In [ ]:
df['RA']=x
df['DEC']=y

In [ ]:
plt.scatter(df['RA'],df['DEC'])
plt.xlabel('RA')
plt.ylabel('DEC')
plt.gca().invert_xaxis()

# 7.Conversion of flux into magnitudes

In [ ]:
df['magVega']= (-2.5*(np.log10(df['Corrected_sum']/header['PHOTMJSR'])))+23.93 # for JWST NIRCAM F335, Zero Point = 23.93 ; change the Z.P according to your data

In [ ]:
plt.hist(df['magVega'])